In [ ]:
import os
import sys


try:
    AUTO_DIR=os.environ["AUTO_DIR"]        
    sys.path.append(os.path.join(AUTO_DIR, "python") )
except KeyError:
    pass


In [ ]:
import auto

 AUTO Demo cusp
===============

Let us study the dynamical system 
\begin{equation}
\frac{d}{dt} x(t) = \mu + \lambda x - x^3
\end{equation}

There is a fixed point at $x_0=0$ for $\lambda_0 = 1.0$, $\mu_0=0$, which we will use as a starting point from which we explore the bifurcations.

Define python variables `unames` and `parnames` which describe the dynamical variables and parameters of the system:


In [ ]:
unames = ["x"]
parnames = ["lambda", "mu"]

# make some dicts for later use
unames = dict(enumerate(unames, start=1))
parnames = dict(enumerate(parnames, start=1))

Note that indices start with `1` in Fortran and with `0` in C. 

Create the `cusp_c.c` file. This contains the enum definitions
```c
enum unames {x};
enum parnames {lambda, mu}; 
```
similar to the corresponding variables in Python. 

The RHS of the dynamical system is then realised through the function

```c
int func(integer ndim, const doublereal *u, const integer *icp, 
         const doublereal *par, integer ijac, doublereal *f, doublereal *dfdu, 
         doublereal *dfdp)
{
    f[x] = par[mu] + par[lambda]*u[x] - u[x]*u[x]*u[x];
    return 0;
} 
```
Finally we have a function which defines the starting point $(x_0; \lambda_0, \mu_0)$

```c
int stpnt(integer ndim, doublereal t, doublereal *u, doublereal *par)
{
    u[x] = 0.0; 
    par[lambda] = 1.0; 
    par[mu] = 0.0; 
    return 0;
} 
```

The remaining functions `pvls()`, `bcnd()`, `icnd()`, and `fopt()` are not required here. They are simply defined as returning zero. 

Unhide the following cell to edit the source code. 

In [ ]:
%%writefile cusp_c.c
#include "auto_f2c.h"

/* ---------------------------------------------------------------------- */
/*   cusp_c.c - cusp normal form                                          */
/* ---------------------------------------------------------------------- */

enum unames {x};
enum parnames {lambda, mu}; 

int func(integer ndim, const doublereal *u, const integer *icp, 
         const doublereal *par, integer ijac, doublereal *f, doublereal *dfdu, 
         doublereal *dfdp)
{
    f[x] = par[mu] + par[lambda]*u[x] - u[x]*u[x]*u[x];
    return 0;
} 

int stpnt(integer ndim, doublereal t, doublereal *u, doublereal *par)
{
    u[x] = 0.0; 
    par[lambda] = 1.0; 
    par[mu] = 0.0; 
    return 0;
} 

int pvls(integer ndim, const doublereal *u, doublereal *par)
{ return 0; } 

int bcnd(integer ndim, const doublereal *par, const integer *icp, integer nbc, 
         const doublereal *u0, const doublereal *u1, integer ijac, doublereal *fb, 
         doublereal *dbc)
{ return 0; } 

int icnd(integer ndim, const doublereal *par, const integer *icp, integer nint, 
         const doublereal *u, const doublereal *uold, const doublereal *udot, 
         const doublereal *upold, integer ijac, doublereal *fi, doublereal *dint)
{ return 0; } 

int fopt(integer ndim, const doublereal *u, const integer *icp, const doublereal *par, 
         integer ijac, doublereal *fs, doublereal *dfdu, doublereal *dfdp)
{ return 0; } 


Load the starting solution.

In [ ]:
cusp = auto.load(e='cusp_c',
                parnames = parnames, 
                unames = unames, 
                NDIM=   len(unames), IPS =   1, IRS =   0, ILP =   1,
                ICP =  ['mu', 'lambda'],
                NTST=   5, NCOL=   4, IAD =   3, ISP =   2, ISW = 1, IPLT= 0, NBC = 0, NINT= 0,
                NMX = 200, NPR =  20, MXBF=   0, IID =   2, ITMX= 8, ITNW= 5, NWTN= 3, JAC = 0,
                EPSL= 1e-06, EPSU = 1e-06, EPSS =0.0001,
                DS  =  0.01, DSMIN= 0.005, DSMAX=   0.1, IADS=   1,
                NPAR = len(parnames), THL =  {}, THU =  {},
                UZSTOP = {'mu': [-2.0, 2.0]}
)

 Run and store the result in the Python variable mu

In [ ]:
mu = auto.run(cusp)

 Run backwards, and append to mu

In [ ]:
mu = mu + auto.run(cusp,DS='-')

 Relabel solutions

In [ ]:
mu = auto.relabel(mu)

 Save to b.mu, s.mu, and d.mu

In [ ]:
auto.save(mu,'mu')

 Plot bifurcation diagram
 
(Unhide the following cell to adjust the plotting defaults).

In [ ]:
%%writefile autorc
[AUTO_plotter]

#default_option="d1"
#d1 = {"grid": "no", "use_labels": 1, "use_symbols": 1, "stability": 1}
# similarly you can redefine d0, d2, d3, d4.

#grid = "no"
#stability = 0
#use_labels = 1
#use_symbols = 1

#top_title = ''
#top_title_fontsize = 12

#xlabel = ''
#xlabel_fontsize = 12
#ylabel = ''
#ylabel_fontsize = 12

#solution_indepvarname = "time"
#solution_coordnames = ["$x$","$y$","$z$"]
#bifurcation_coordnames = ["?", "L2-norm"]

#line_width = 2.0
#dashes = (6.0,6.0)
#background = "white"
#foreground = "black"
#color_list = "black red green blue"
#symbol_color = "red"
#symbol_font = "-misc-fixed-*-*-*-*-*-*-*-*-*-*-*-*"
#decorations = 1
#smart_label = 1
#minx = 0
#maxx = 0
#miny = 0
#maxy = 0
#width = 600
#height = 480
#left_margin = 80
#right_margin = 40
#top_margin = 40
#bottom_margin = 40
#xticks = 5
#yticks = 5
#tick_label_template = "%.2e"
#tick_length = 0.2
#odd_tick_length = 0.4
#even_tick_length = 0.2
#ps_colormode = "color"
#mark_t = None #or a real value between 0 and 1

#type = "bifurcation" # or "solution"

#bifurcation_x = [0]
#bifurcation_y = [1]

#solution_x = ["t"]
#solution_y = [0]

## Sets of columns that the user is likely to want to use
#bifurcation_column_defaults = None
#solution_column_defaults = None

# The label(s) of the solution we wish to draw
#label = [1,2,3]
#label_defaults = None
# The index/indices of the solution we wish to draw
#index = [0]

#bifurcation_diagram_filename = 'fort.7'
#solution_filename = 'fort.8'

#bifurcation_symbol = "square"
#limit_point_symbol = None
#hopf_symbol = "fillsquare"
#period_doubling_symbol = "doubletriangle"
#torus_symbol = "filldiamond"
#user_point_symbol = "U"
#error_symbol = None



In [ ]:
p = auto.plot(mu)
p.config(bifurcation_y=['x'])

 Set the new start label to the first LP label in b.mu and s.mu

In [ ]:
lp1 = auto.load(mu('LP1'), ISW=2)

 Continue from this label in two parameters

In [ ]:
cusp = auto.run(lp1)
cusp = cusp + auto.run(lp1,DS='-')

 save to b.cusp, s.cusp, and d.cusp

In [ ]:
auto.save(cusp,'cusp')

 Plot the cusp

In [ ]:
p = auto.plot(cusp)
p.config(bifurcation_y=['lambda'])

In [ ]:
# Stop automatic execution at this point
assert(False)

clean the directory

In [ ]:
# comment out to clean directory
auto.delete("mu")
auto.delete("cusp")
auto.clean()
#!rm -f cusp_c.c autorc

## Use low level interface

to work with less "state" use the low-level interface.  The following might not be guaranteed to work as it is not officially documented in the handbook. Use with care. 

We define `unames` and `parnames` as before, and additionally collect all constants for our system in a `constants` variable. 

In [ ]:
unames = ["x"]
parnames = ["lambda", "mu"]

unames = dict(enumerate(unames, start=1))
parnames = dict(enumerate(parnames, start=1))

constants = auto.parseC.parseC(
    e = "cusp_c",
    unames = unames, parnames = parnames, 
    NDIM = len(unames), NPAR = len(parnames),

    NPR = 20, # print after NPR points
    NMX = 200, # max number of points per run
    MXBF = 0, # Don't follow bifus automatically
    UZSTOP = {'mu': [-2.0, 2.0]}
)

Here we refer to a `"cusp_c"` engine, which is defined almost as before. The only difference is that we now will not require the `stpnt()` function, as we will later define the starting point in Python. (Unhide to edit)

In [ ]:
%%writefile cusp_c.c
#include "auto_f2c.h"

/* ---------------------------------------------------------------------- */
/*   cusp_c.c - cusp normal form                                          */
/* ---------------------------------------------------------------------- */

enum unames {x};
enum parnames {lambda, mu}; 

int func(integer ndim, const doublereal *u, const integer *icp, 
         const doublereal *par, integer ijac, doublereal *f, doublereal *dfdu, 
         doublereal *dfdp)
{
    f[x] = par[mu] + par[lambda]*u[x] - u[x]*u[x]*u[x];
    return 0;
} 

int stpnt(integer ndim, doublereal t, doublereal *u, doublereal *par)
{ return 0; }

int pvls(integer ndim, const doublereal *u, doublereal *par)
{ return 0; } 

int bcnd(integer ndim, const doublereal *par, const integer *icp, integer nbc, 
         const doublereal *u0, const doublereal *u1, integer ijac, doublereal *fb, 
         doublereal *dbc)
{ return 0; } 

int icnd(integer ndim, const doublereal *par, const integer *icp, integer nint, 
         const doublereal *u, const doublereal *uold, const doublereal *udot, 
         const doublereal *upold, integer ijac, doublereal *fi, doublereal *dint)
{ return 0; } 

int fopt(integer ndim, const doublereal *u, const integer *icp, const doublereal *par, 
         integer ijac, doublereal *fs, doublereal *dfdu, doublereal *dfdp)
{ return 0; } 


The starting point is defined through:

In [ ]:
u0 = [0.0]
par0 = {"lambda": 1.0, "mu": 0.0}
sol0 = auto.parseS.AUTOSolution(u0, PAR=par0, constants=constants)

We do not want to depend on the global "runner" which is responsible for much of the state introduced between different calls to `auto.run()`.  Instead we use `AUTOSolution.run()` which creates a fresh runner each time we run it. 

In [ ]:
mu = sol0.run(ICP = ['mu', 'lambda'])

 Run backwards, and append to mu

In [ ]:
mu = mu + sol0.run(DS='-',ICP = ['mu', 'lambda'])

 Relabel solutions

In [ ]:
mu = mu.relabel()

 Plot bifurcation diagram (this uses the `autorc` file from before)

In [ ]:
p = auto.plot(mu)
p.config(bifurcation_y=['x'])

 Set the new start label to the first LP label in b.mu and s.mu

In [ ]:
lp1 = mu("LP1")

Continue from this label in two parameters

In [ ]:
cusp = lp1.run(ISW=2) + lp1.run(ISW=2, DS="-")
cusp = cusp.relabel()

 Plot the cusp

In [ ]:
p = auto.plot(cusp)
p.config(bifurcation_y=['lambda'])
p.config(minx=-2, maxx=2, miny=0, maxy=3)

In [ ]:
# Stop automatic execution at this point
assert(False)

In [ ]:
# comment out to clean directory
auto.clean()
!rm -f cusp_c.c 